<a href="https://colab.research.google.com/github/a-ting-j/LLMFine-tuningExperiment/blob/main/nb/Qwen3_(4B)-GRPO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 安装需要的包

In [1]:
%%capture
import os
os.environ["UNSLOTH_VLLM_STANDBY"] = "1" # 多出30%的上下文长度
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm

In [2]:
#@title 在Colab上的额外安装 { display-mode: "form" }
%%capture
import os
!pip install --upgrade -qqq uv
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    try: import numpy, PIL; get_numpy = f"numpy=={numpy.__version__}"; get_pil = f"pillow=={PIL.__version__}"
    except: get_numpy = "numpy"; get_pil = "pillow"
    try: import subprocess; is_t4 = "Tesla T4" in str(subprocess.check_output(["nvidia-smi"]))
    except: is_t4 = False
    get_vllm, get_triton = ("vllm==0.9.2", "triton==3.2.0") if is_t4 else ("vllm==0.10.2", "triton")
    !uv pip install -qqq --upgrade \
        unsloth {get_vllm} {get_numpy} {get_pil} torchvision bitsandbytes xformers
    !uv pip install -qqq {get_triton}
!uv pip install transformers==4.56.2
!uv pip install --no-deps trl==0.22.2

### Unsloth

目标: 使用 OpenR1 的 Math 数据集，通过 GRPO 将 `Llama` 转换为推理模型。

我们首先对模型进行预微调，使 GRPO 跳过尝试匹配格式的步骤——这可以加快 GRPO 的速度。

In [3]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # 可以增加更长的推理轨迹
lora_rank = 32 # 等级越高 = 越聪明，但速度越慢。建议值：8、16、32、64、128

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-1B-Instruct-bnb-4bit",
    max_seq_length = max_seq_length,
    load_in_4bit = False, # False for LoRA 16bit
    fast_inference = True, # 启用 vLLM 快速推理
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.9, # 内存使用比
)

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank,
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha = lora_rank*2, # 两倍的训练速度
    use_gradient_checkpointing = "unsloth", # 减少内存使用
    random_state = 3407,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/usr/local/lib/python3.12/dist-packages/torchao/quantization/quant_api.py:2525: SyntaxWarning: invalid escape sequence '\.'
  * regex for parameter names, must start with `re:`, e.g. `re:language\.layers\..+\.q_proj.weight`.


INFO 12-22 12:38:07 [__init__.py:244] Automatically detected platform cuda.
ERROR 12-22 12:38:09 [fa_utils.py:57] Cannot use FA version 2 is not supported due to FA2 is only supported on devices with compute capability >= 8
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 12-22 12:38:30 [vllm_utils.py:702] Unsloth: Patching vLLM v1 graph capture
INFO 12-22 12:38:30 [vllm_utils.py:732] Unsloth: Patching vLLM v0 graph capture
==((====))==  Unsloth 2025.12.8: Fast Llama patching. Transformers: 4.56.2. vLLM: 0.9.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Standby mode is enabled. Changing `gpu_memory_utilization` to 0.9.
Unsloth: vL

`torch_dtype` is deprecated! Use `dtype` instead!


WARNING 12-22 12:38:50 [config.py:3371] Casting torch.bfloat16 to torch.float16.
INFO 12-22 12:38:50 [config.py:1472] Using max model len 2048
WARNING 12-22 12:38:50 [arg_utils.py:1735] Compute Capability < 8.0 is not supported by the V1 Engine. Falling back to V0. 
INFO 12-22 12:38:53 [config.py:2285] Chunked prefill is enabled with max_num_batched_tokens=4096.
INFO 12-22 12:38:53 [llm_engine.py:230] Initializing a V0 LLM engine (v0.9.2) with config: model='unsloth/Llama-3.2-1B-Instruct', speculative_config=None, tokenizer='unsloth/Llama-3.2-1B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config={}, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=2048, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(backend='auto'

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

Unsloth 2025.12.8 patched 16 layers with 16 QKV layers, 16 O layers and 16 MLP layers.


### GRPO 对话模版
由于我们使用的是基础模型，因此需要设置聊天模板。您也可以创建自己的聊天模板！

1. DeepSeek 使用了 `<think>` 和 `</think>`，但这并非必需——您可以根据自己的喜好进行自定义！ llama使用的是隐示思考。

2. 建议使用 `system_prompt` 来引导模型的响应。

In [4]:
reasoning_start = "<start_working_out>" # 作用等同于 <think>
reasoning_end   = "<end_working_out>"   # 作用等同于 </think>
solution_start  = "<SOLUTION>"
solution_end    = "</SOLUTION>"

system_prompt = \
f"""You are given a problem.
Think about the problem and provide your working out.
Place it between {reasoning_start} and {reasoning_end}.
Then, provide your solution between {solution_start}{solution_end}"""
system_prompt

'You are given a problem.\nThink about the problem and provide your working out.\nPlace it between <start_working_out> and <end_working_out>.\nThen, provide your solution between <SOLUTION></SOLUTION>'

我们在下面创建了一个简单的聊天模板。请注意，`add_generation_prompt` 包含前缀 `<start_working_out>`，用于引导模型开始其推理过程。

In [5]:
chat_template = \
    "{% if messages[0]['role'] == 'system' %}"\
        "{{ messages[0]['content'] + eos_token }}"\
        "{% set loop_messages = messages[1:] %}"\
    "{% else %}"\
        "{{ '{system_prompt}' + eos_token }}"\
        "{% set loop_messages = messages %}"\
    "{% endif %}"\
    "{% for message in loop_messages %}"\
        "{% if message['role'] == 'user' %}"\
            "{{ message['content'] }}"\
        "{% elif message['role'] == 'assistant' %}"\
            "{{ message['content'] + eos_token }}"\
        "{% endif %}"\
    "{% endfor %}"\
    "{% if add_generation_prompt %}{{ '{reasoning_start}' }}"\
    "{% endif %}"

# Replace with out specific template:
chat_template = chat_template\
    .replace("'{system_prompt}'",   f"'{system_prompt}'")\
    .replace("'{reasoning_start}'", f"'{reasoning_start}'")
tokenizer.chat_template = chat_template

让我们通过一个示例来看看我们的聊天模板是如何运行的：

In [6]:
tokenizer.apply_chat_template([
    {"role" : "user", "content" : "What is 1+1?"},
    {"role" : "assistant", "content" : f"{reasoning_start}I think it's 2.{reasoning_end}{solution_start}2{solution_end}"},
    {"role" : "user", "content" : "What is 2+2?"},
], tokenize = False, add_generation_prompt = True)

"You are given a problem.\nThink about the problem and provide your working out.\nPlace it between <start_working_out> and <end_working_out>.\nThen, provide your solution between <SOLUTION></SOLUTION><|eot_id|>What is 1+1?<start_working_out>I think it's 2.<end_working_out><SOLUTION>2</SOLUTION><|eot_id|>What is 2+2?<start_working_out>"

### 预微调格式

我们现在使用 NVIDIA 的 [Open Math Reasoning 数据集](https://huggingface.co/datasets/nvidia/OpenMathReasoning) 的一个子集，该子集经过筛选，仅包含高质量的 DeepSeek R1 轨迹。

我们将仅筛选约 59 个样本，以首先“初始化”/预微调模型，使其能够理解我们自定义的 GRPO 格式。

In [7]:
from datasets import load_dataset
import pandas as pd
import numpy as np

dataset = load_dataset("unsloth/OpenMathReasoning-mini", split = "cot")
dataset = dataset.to_pandas()[
    ["expected_answer", "problem", "generated_solution"]
]

# Try converting to number - if not, replace with NaN
is_number = pd.to_numeric(pd.Series(dataset["expected_answer"]), errors = "coerce").notnull()
# Select only numbers
dataset = dataset.iloc[np.where(is_number)[0]]

dataset

README.md:   0%|          | 0.00/603 [00:00<?, ?B/s]

data/cot-00000-of-00001.parquet:   0%|          | 0.00/106M [00:00<?, ?B/s]

Generating cot split:   0%|          | 0/19252 [00:00<?, ? examples/s]

,expected_answer,problem,generated_solution
0,14,Given $\sqrt{x^2+165}-\sqrt{x^2-52}=7$ and $x$...,"<think>\nOkay, let's see. I need to solve the ..."
6,-2,Find the value of the parameter $a$ for which ...,"<think>\nOkay, so I need to find the value of ..."
9,18,What is the sum of all real numbers $x$ for wh...,"<think>\nOkay, so I need to solve the equation..."
13,2,Evaluate the sum \(\sum_{n=1}^\infty \frac{\ph...,"<think>\nOkay, so I need to evaluate the infin..."
17,30,What is the largest positive integer that divi...,"<think>\nAlright, so I need to find the larges..."
...,...,...,...
19243,244,"Let \( p \), \( q \), and \( r \) be the disti...","<think>\nOkay, so I need to find the value of ..."
19245,1,A bug is on the $0$ of a number line. At any p...,"<think>\nOkay, so I have this problem where a ..."
19247,4,A bus left point X for point Y. Two hours late...,"<think>\nOkay, let's tackle this problem step ..."
19248,18,Each interior angle of a regular n-gon measure...,"<think>\nOkay, let's see. I need to find the n..."


我们需要将数据集格式化为符合 GRPO 风格的格式：

In [8]:
def format_dataset(x):
    expected_answer = x["expected_answer"]
    problem = x["problem"]

    # Remove generated <think> and </think>
    thoughts = x["generated_solution"]
    thoughts = thoughts.replace("<think>", "").replace("</think>", "")

    # Strip newlines on left and right
    thoughts = thoughts.strip()
    # Add our custom formatting
    final_prompt = \
        reasoning_start + thoughts + reasoning_end + \
        solution_start + expected_answer + solution_end
    return [
        {"role" : "system",    "content" : system_prompt},
        {"role" : "user",      "content" : problem},
        {"role" : "assistant", "content" : final_prompt},
    ]

dataset["Messages"] = dataset.apply(format_dataset, axis = 1)

检查一下是否有效：

In [9]:
tokenizer.apply_chat_template(dataset["Messages"][0], tokenize = False)

"You are given a problem.\nThink about the problem and provide your working out.\nPlace it between <start_working_out> and <end_working_out>.\nThen, provide your solution between <SOLUTION></SOLUTION><|eot_id|>Given $\\sqrt{x^2+165}-\\sqrt{x^2-52}=7$ and $x$ is positive, find all possible values of $x$.<start_working_out>Okay, let's see. I need to solve the equation √(x² + 165) - √(x² - 52) = 7, and find all positive values of x. Hmm, radicals can be tricky, but maybe if I can eliminate the square roots by squaring both sides. Let me try that.\n\nFirst, let me write down the equation again to make sure I have it right:\n\n√(x² + 165) - √(x² - 52) = 7.\n\nOkay, so the idea is to isolate one of the radicals and then square both sides. Let me try moving the second radical to the other side:\n\n√(x² + 165) = 7 + √(x² - 52).\n\nNow, if I square both sides, maybe I can get rid of the square roots. Let's do that:\n\n(√(x² + 165))² = (7 + √(x² - 52))².\n\nSimplifying the left side:\n\nx² + 165

我们将微调前的数据集截断为 `max_seq_length/2`，因为我们不希望推理过程的轨迹过长。

注意，这可能需要 2 分钟！

In [10]:
dataset["N"] = dataset["Messages"].apply(lambda x: len(tokenizer.apply_chat_template(x)))

dataset = dataset.loc[dataset["N"] <= max_seq_length/2].copy()
dataset.shape

(78, 5)

然后我们对消息进行分词，并将其转换为与 Hugging Face 兼容的数据集格式：

In [11]:
from datasets import Dataset

dataset["text"] = tokenizer.apply_chat_template(dataset["Messages"].values.tolist(), tokenize = False)
dataset = Dataset.from_pandas(dataset)
dataset

Dataset({
    features: ['expected_answer', 'problem', 'generated_solution', 'Messages', 'N', 'text', '__index_level_0__'],
    num_rows: 78
})

现在让我们预先微调模型，使其遵循我们自定义的 GRPO 格式！

In [12]:
from trl import SFTTrainer, SFTConfig
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 1, # Use GA to mimic batch size!
        warmup_steps = 5,
        num_train_epochs = 2, # Set this for 1 full training run.
        learning_rate = 2e-4, # Reduce to 2e-5 for long training runs
        logging_steps = 5,
        optim = "adamw_8bit",
        weight_decay = 0.001,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "none", # Use TrackIO/WandB etc
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/78 [00:00<?, ? examples/s]

🦥 Unsloth: Padding-free auto-enabled, enabling faster training.


In [13]:
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 78 | Num Epochs = 2 | Total steps = 156
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 1 x 1) = 1
 "-____-"     Trainable parameters = 22,544,384 of 1,258,358,784 (1.79% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
5,1.238400
10,0.955300
15,0.777800
20,0.817400
25,0.728000
30,0.741200
35,0.747900
40,0.706900
45,0.644700
50,0.726200


TrainOutput(global_step=156, training_loss=0.5855087989415878, metrics={'train_runtime': 77.4499, 'train_samples_per_second': 2.014, 'train_steps_per_second': 2.014, 'total_flos': 836105206063104.0, 'train_loss': 0.5855087989415878, 'epoch': 2.0})

我们检查一下模型是否已经学会遵循自定义格式：

In [14]:
text = tokenizer.apply_chat_template(
    dataset[0]["Messages"][:2],
    tokenize = False,
    add_generation_prompt = True, # Must add for generation
)

from transformers import TextStreamer
_ = model.generate(
    **tokenizer(text, return_tensors = "pt").to("cuda"),
    temperature = 0,
    max_new_tokens = 1024,
    streamer = TextStreamer(tokenizer, skip_prompt = False),
)

<|begin_of_text|>You are given a problem.
Think about the problem and provide your working out.
Place it between <start_working_out> and <end_working_out>.
Then, provide your solution between <SOLUTION></SOLUTION><|eot_id|>Jenifer has 82 cents in pennies and nickels. Her younger brother mistook all her nickels for dimes and counted the total as $1.47. How many pennies does Jenifer have?<start_working_out>Okay, let's see. So the problem is about Jenifer who has 82 cents in pennies and nickels. Her brother thought all the nickels were dimes and counted the total as $1.47. We need to find out how many pennies Jenifer has. Hmm, let's break this down step by step.

First, let me remember that a nickel is worth 5 cents and a dime is worth 10 cents. So if her brother thought nickels were dimes, then he counted the total as if all nickels were worth 10 cents each. Let me denote the number of nickels as N. Then the total amount she had would be 5N cents. Since her brother counted it as $1.47, w

格式符合要求！我们在 GRPO 步骤之前删除一些项目。

In [15]:
del dataset
torch.cuda.empty_cache()
import gc
gc.collect()

3024

### 数据准备
<a name="Data"></a>

我们使用的是 Hugging Face 的 [Open R1 Math 数据集](https://huggingface.co/datasets/open-r1/DAPO-Math-17k-Processed)。您也可以使用 OpenAI 著名的 [GSM8K 数据集](https://huggingface.co/datasets/openai/gsm8k)。

In [16]:
from datasets import load_dataset
dataset = load_dataset("open-r1/DAPO-Math-17k-Processed", "en", split = "train")
dataset

README.md: 0.00B [00:00, ?B/s]

en/train-00000-of-00001.parquet:   0%|          | 0.00/5.23M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/14116 [00:00<?, ? examples/s]

Dataset({
    features: ['prompt', 'solution', 'data_source', 'source_prompt', 'ability', 'reward_model', 'extra_info'],
    num_rows: 14116
})

我们看一下第一行：

In [17]:
dataset[0]["prompt"]

'In triangle $ABC$, $\\sin \\angle A = \\frac{4}{5}$ and $\\angle A < 90^\\circ$. Let $D$ be a point outside triangle $ABC$ such that $\\angle BAD = \\angle DAC$ and $\\angle BDC = 90^\\circ$. Suppose that $AD = 1$ and that $\\frac{BD}{CD} = \\frac{3}{2}$. If $AB + AC$ can be expressed in the form $\\frac{a\\sqrt{b}}{c}$ where $a, b, c$ are pairwise relatively prime integers, find $a + b + c$.'

In [18]:
dataset[0]["solution"]

'34'

在 GSM8K 数据集中，我们注意到所有类似“关于”的回答都带有 ####，因此我们将其提取出来。但对于 Open R1 数据集，我们可以跳过以下步骤。

In [19]:
def extract_hash_answer(text):
    # if "####" not in text: return None
    # return text.split("####")[1].strip()
    return text
extract_hash_answer(dataset[0]["solution"])

'34'

我们来绘制数据集！看看第一行：

In [20]:
dataset = dataset.map(lambda x: {
    "prompt" : [
        {"role": "system", "content": system_prompt},
        {"role": "user",   "content": x["prompt"]},
    ],
    "answer": extract_hash_answer(x["solution"]),
})
dataset[0]

Map:   0%|          | 0/14116 [00:00<?, ? examples/s]

{'prompt': [{'content': 'You are given a problem.\nThink about the problem and provide your working out.\nPlace it between <start_working_out> and <end_working_out>.\nThen, provide your solution between <SOLUTION></SOLUTION>',
   'role': 'system'},
  {'content': 'In triangle $ABC$, $\\sin \\angle A = \\frac{4}{5}$ and $\\angle A < 90^\\circ$. Let $D$ be a point outside triangle $ABC$ such that $\\angle BAD = \\angle DAC$ and $\\angle BDC = 90^\\circ$. Suppose that $AD = 1$ and that $\\frac{BD}{CD} = \\frac{3}{2}$. If $AB + AC$ can be expressed in the form $\\frac{a\\sqrt{b}}{c}$ where $a, b, c$ are pairwise relatively prime integers, find $a + b + c$.',
   'role': 'user'}],
 'solution': '34',
 'data_source': 'math_dapo',
 'source_prompt': [{'content': 'Solve the following math problem step by step. The last line of your response should be of the form Answer: $Answer (without quotes) where $Answer is the answer to the problem.\n\nIn triangle $ABC$, $\\sin \\angle A = \\frac{4}{5}$ and $

我们创建了一个正则表达式格式，用于匹配推理部分和答案：

In [21]:
import re

# Add optional EOS token matching
solution_end_regex = r"</SOLUTION>[\s]{0,}" + \
    "(?:" + re.escape(tokenizer.eos_token) + ")?"

match_format = re.compile(
    rf"{reasoning_end}.*?"\
    rf"{solution_start}(.+?){solution_end_regex}"\
    rf"[\s]{{0,}}$",
    flags = re.MULTILINE | re.DOTALL
)
match_format

re.compile(r'<end_working_out>.*?<SOLUTION>(.+?)</SOLUTION>[\s]{0,}(?:<\|eot_id\|>)?[\s]{0,}$',
re.MULTILINE|re.DOTALL|re.UNICODE)

我们验证了它的有效性：

In [22]:
match_format.findall(
    "Let me think!<end_working_out>"\
    f"<SOLUTION>\n2\n</SOLUTION>",
)

['\n2\n']

In [23]:
match_format.findall(
    "<start_working_out>Let me think!<end_working_out>"\
    f"<SOLUTION>  2  </SOLUTION>\n\n",
)

['  2  ']

现在我们想创建一个与格式完全匹配的奖励函数——如果成功，我们奖励它 3 分：

In [24]:
def match_format_exactly(completions, **kwargs):
    scores = []
    for completion in completions:
        score = 0
        response = completion[0]["content"]
        # Match if format is seen exactly!
        if match_format.search(response) is not None: score += 3.0
        scores.append(score)
    return scores

如果失败，我们希望通过统计每个符号的数量，来奖励至少部分符合格式的模型：

In [25]:
def match_format_approximately(completions, **kwargs):
    scores = []
    for completion in completions:
        score = 0
        response = completion[0]["content"]
        # Count how many keywords are seen - we penalize if too many!
        # If we see 1, then plus some points!

        # No need to reward <start_working_out> since we always prepend it!
        # score += 0.5 if response.count(reasoning_start) == 1 else -1.0
        score += 0.5 if response.count(reasoning_end)   == 1 else -1.0
        score += 0.5 if response.count(solution_start)  == 1 else -1.0
        score += 0.5 if response.count(solution_end)    == 1 else -1.0
        scores.append(score)
    return scores

最后，我们要提取生成的答案，并对其进行奖励或惩罚！我们还会根据答案与真实答案的接近程度，通过比例来给予奖励：

In [26]:
def check_answer(prompts, completions, answer, **kwargs):
    question = prompts[0][-1]["content"]
    responses = [completion[0]["content"] for completion in completions]

    extracted_responses = [
        guess.group(1)
        if (guess := match_format.search(r)) is not None else None \
        for r in responses
    ]

    scores = []
    for guess, true_answer in zip(extracted_responses, answer):
        score = 0
        if guess is None:
            scores.append(-2.0)
            continue
        # Correct answer gets 5 points!
        if guess == true_answer:
            score += 5.0
        # Match if spaces are seen, but less reward
        elif guess.strip() == true_answer.strip():
            score += 3.5
        else:
            # We also reward it if the answer is close via ratios!
            # Ie if the answer is within some range, reward it!
            try:
                ratio = float(guess) / float(true_answer)
                if   ratio >= 0.9 and ratio <= 1.1: score += 2.0
                elif ratio >= 0.8 and ratio <= 1.2: score += 1.5
                else: score -= 2.5 # Penalize wrong answers
            except:
                score -= 4.5 # Penalize
        scores.append(score)
    return scores

有时答案可能不是一个数字，而是一个句子，例如“答案是 20 美元”——我们提取 20。

我们还会移除逗号，例如 123,456。

In [27]:
match_numbers = re.compile(
    solution_start + r".*?[\s]{0,}([-]?[\d\.\,]{1,})",
    flags = re.MULTILINE | re.DOTALL
)
print(match_numbers.findall("<SOLUTION>  0.34  </SOLUTION>"))
print(match_numbers.findall("<SOLUTION>  123,456  </SOLUTION>"))
print(match_numbers.findall("<SOLUTION>  -0.234  </SOLUTION>"))
print(match_numbers.findall("<SOLUTION>17</SOLUTION>"))

['0.34']
['123,456']
['-0.234']
['17']


现在我们准备主函数，该函数将打印出生成的响应和正确答案，以及另一个奖励函数，该函数通过 `float` 将文本转换为浮点数，并查看它们是否相同。

In [28]:
global PRINTED_TIMES
PRINTED_TIMES = 0
global PRINT_EVERY_STEPS
PRINT_EVERY_STEPS = 5

def check_numbers(prompts, completions, answer, **kwargs):
    question = prompts[0][-1]["content"]
    responses = [completion[0]["content"] for completion in completions]

    extracted_responses = [
        guess.group(1)
        if (guess := match_numbers.search(r)) is not None else None \
        for r in responses
    ]

    scores = []
    # Print only every few steps
    global PRINTED_TIMES
    global PRINT_EVERY_STEPS
    if PRINTED_TIMES % PRINT_EVERY_STEPS == 0:
        print(
            '*'*20 + f"Question:\n{question}", f"\nAnswer:\n{answer[0]}", f"\nResponse:\n{responses[0]}", f"\nExtracted:\n{extracted_responses[0]}"
        )
    PRINTED_TIMES += 1

    for guess, true_answer in zip(extracted_responses, answer):
        if guess is None:
            scores.append(-2.5)
            continue
        # Convert to numbers
        try:
            true_answer = float(true_answer.strip())
            # Remove commas like in 123,456
            guess       = float(guess.strip().replace(",", ""))
            scores.append(3.5 if guess == true_answer else -1.5)
        except:
            scores.append(0)
            continue
    return scores

获取长度排名前 90% 的提示信息，以免意外截断！

也就是说，我们会移除长度排名后 10% 的提示信息。

In [29]:
tokenized = dataset.map(
    lambda x: {"tokens" : tokenizer.apply_chat_template(x["prompt"], add_generation_prompt = True, tokenize = True)},
    batched = True,
)
print(tokenizer.decode(tokenized[0]["tokens"]))
tokenized = tokenized.map(lambda x: {"L" : len(x["tokens"])})

import numpy as np
maximum_length = int(np.quantile(tokenized["L"], 0.9))
print("Max Length = ", maximum_length)

# Filter only samples smaller than 90% max length
dataset = dataset.select(np.where(np.array(tokenized["L"]) <= maximum_length)[0])
del tokenized

Map:   0%|          | 0/14116 [00:00<?, ? examples/s]

You are given a problem.
Think about the problem and provide your working out.
Place it between <start_working_out> and <end_working_out>.
Then, provide your solution between <SOLUTION></SOLUTION><|eot_id|>In triangle $ABC$, $\sin \angle A = \frac{4}{5}$ and $\angle A < 90^\circ$. Let $D$ be a point outside triangle $ABC$ such that $\angle BAD = \angle DAC$ and $\angle BDC = 90^\circ$. Suppose that $AD = 1$ and that $\frac{BD}{CD} = \frac{3}{2}$. If $AB + AC$ can be expressed in the form $\frac{a\sqrt{b}}{c}$ where $a, b, c$ are pairwise relatively prime integers, find $a + b + c$.<start_working_out>


Map:   0%|          | 0/14116 [00:00<?, ? examples/s]

Max Length =  198


<a name="Train"></a>
### Train the model

现在设置 GRPO Trainer 并进行所有配置！

In [30]:
max_prompt_length = maximum_length + 1 # + 1 just in case!
max_completion_length = max_seq_length - max_prompt_length

from vllm import SamplingParams
vllm_sampling_params = SamplingParams(
    min_p = 0.1,
    top_p = 1.0,
    top_k = -1,
    seed = 3407,
    stop = [tokenizer.eos_token],
    include_stop_str_in_output = True,
)

from trl import GRPOConfig, GRPOTrainer
training_args = GRPOConfig(
    vllm_sampling_params = vllm_sampling_params,
    temperature = 1.0,
    learning_rate = 5e-6,
    weight_decay = 0.001,
    warmup_ratio = 0.1,
    lr_scheduler_type = "linear",
    # optim = "adamw_8bit",
    optim="adamw_torch_fused",
    logging_steps = 1,
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 16, # Increase to 4 for smoother training
    num_generations = 4, # Decrease if out of memory
    max_prompt_length = max_prompt_length,
    max_completion_length = max_completion_length,
    # num_train_epochs = 1, # Set to 1 for a full training run
    max_steps = 100,
    save_steps = 100,
    report_to = "none", # Can use Weights & Biases
    output_dir = "outputs",

    # For optional training + evaluation
    # fp16_full_eval = True,
    # per_device_eval_batch_size = 4,
    # eval_accumulation_steps = 1,
    # eval_strategy = "steps",
    # eval_steps = 1,
)

现在开始运行训练器！向上滚动，你会看到一个奖励表格。目标是让“reward”列的值增加！

你可能需要等待 150 到 200 步才能触发任何操作。前 100 步可能不会获得任何奖励。请耐心等待！

| Step | Training Loss | reward    | reward_std | completion_length | kl       |
|------|---------------|-----------|------------|-------------------|----------|
| 1    | 0.000000      | 0.125000  | 0.000000   | 200.000000        | 0.000000 |
| 2    | 0.000000      | 0.072375  | 0.248112   | 200.000000        | 0.000000 |
| 3    | 0.000000      | -0.079000 | 0.163776   | 182.500000        | 0.000005 |


In [31]:
# For optional training + evaluation
# new_dataset = dataset.train_test_split(test_size = 0.01)

trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [
        match_format_exactly,
        match_format_approximately,
        check_answer,
        check_numbers,
    ],
    args = training_args,
    train_dataset = dataset,

    # For optional training + evaluation
    # train_dataset = new_dataset["train"],
    # eval_dataset = new_dataset["test"],
)
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 12,715 | Num Epochs = 1 | Total steps = 100
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 16
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 16 x 1) = 16
 "-____-"     Trainable parameters = 22,544,384 of 1,258,358,784 (1.79% trained)


********************Question:
A conical glass is in the form of a right circular cone. The slant height is $21$ and the radius of the top rim of the glass is $14$. An ant at the midpoint of a slant line on the outside wall of the glass sees a honey drop diametrically opposite to it on the inside wall of the glass. If $d$ is the shortest distance it should crawl to reach the honey drop, what is the integer part of $d$? 
Answer:
18 
Response:
Okay, let's see. So there's this right circular cone, right? The cone is glass, right? And there's a honey drop inside that's diametrically opposite on the inside wall. The question is asking for the shortest distance, d, that an ant has to crawl, where d is the distance from the midpoint of a slant line on the outside wall to the honey drop.

First, I need to remember the dimensions given. The slant height is 21 units. The radius of the top rim is 14 units. Wait, but I need the radius of the base of the cone. Because the problem is about a cone wit

Step,Training Loss,reward,reward_std,completions / mean_length,completions / min_length,completions / max_length,completions / clipped_ratio,completions / mean_terminated_length,completions / min_terminated_length,completions / max_terminated_length,kl,rewards / match_format_exactly / mean,rewards / match_format_exactly / std,rewards / match_format_approximately / mean,rewards / match_format_approximately / std,rewards / check_answer / mean,rewards / check_answer / std,rewards / check_numbers / mean,rewards / check_numbers / std
1,0.000300,-2.031250,3.477078,1619.125000,963.000000,1849.000000,0.562500,1323.571533,963.000000,1561.000000,0.323402,1.312500,1.537043,-0.281250,2.205816,-1.500000,1.974842,-1.562500,1.436141
2,0.000300,-3.500000,4.568914,1678.500000,792.000000,1849.000000,0.562500,1459.285767,792.000000,1779.000000,0.341252,1.312500,1.537043,-1.031250,2.305564,-1.718750,1.277612,-2.062500,0.512348
3,0.000300,-6.125000,2.750000,1771.562500,1308.000000,1849.000000,0.812500,1436.000000,1308.000000,1570.000000,0.293771,0.562500,1.209339,-2.156250,1.814008,-2.218750,0.631631,-2.312500,0.403113
4,0.000300,-6.500000,2.000000,1837.687500,1755.000000,1849.000000,0.875000,1758.500000,1755.000000,1762.000000,0.289642,0.375000,1.024695,-2.437500,1.537043,-2.062500,0.170783,-2.375000,0.341565
5,0.000400,-2.000000,3.913006,1623.125000,887.000000,1849.000000,0.375000,1487.599976,887.000000,1815.000000,0.358510,1.875000,1.500000,-0.187500,2.250000,-1.812500,1.314978,-1.875000,0.500000
6,0.000300,-2.437500,5.172111,1656.937500,972.000000,1849.000000,0.500000,1464.875000,972.000000,1775.000000,0.287873,1.500000,1.549193,-0.750000,2.323790,-1.812500,1.611159,-1.375000,1.962142
7,0.000300,-5.375000,2.896802,1682.125000,1098.000000,1849.000000,0.687500,1315.000000,1098.000000,1564.000000,0.292910,0.937500,1.436141,-1.593750,2.154211,-2.531250,0.991106,-2.187500,0.478714
8,0.000300,-2.093750,3.918193,1524.125000,919.000000,1849.000000,0.375000,1329.200073,919.000000,1823.000000,0.277257,1.687500,1.537043,-0.281250,2.205816,-1.937500,1.948290,-1.562500,1.436141
9,0.000400,-3.000000,5.199610,1517.125000,974.000000,1849.000000,0.500000,1185.250000,974.000000,1393.000000,0.361153,1.500000,1.549193,-0.656250,2.256241,-2.156250,2.134782,-1.687500,1.470544
10,0.000300,-6.000000,3.000000,1767.312500,1278.000000,1849.000000,0.812500,1413.333374,1278.000000,1503.000000,0.263243,0.562500,1.209339,-2.156250,1.814008,-2.093750,0.201556,-2.312500,0.403113


********************Question:
Find the last three digits of the product of the positive roots of $\sqrt{1995}x^{\log_{1995}x}=x^2$ . 
Answer:
25 
Response:
Okay, so I need to find the last three digits of the product of the positive roots of this equation. Alright, let's see. The equation is sqrt(1995)*x^(log(1995)*x) = x^2. We are looking for the product of the positive roots. Hmm, log_{1995}x here. Wait, what does that even mean? It's a base of 1995 root. Let me rationalize that.

First, let me rewrite log_{1995}x. Oh, right, the logarithm to a certain base. Usually, the base is 10. Wait, maybe I'm missing something. But wait, log is a base 10 function by definition. Wait, but here it's log_{1995}x. That's a different story because the base is 1995. So, maybe the equation is x^((log_{1995}x)/0.5) = x^2? Let me check that again. If log base 1995 of x, then 2000. So log_{1995}x is 0.2. Then the equation becomes x^(0.2) = x^2. Because 0.5 times 0.2 is 0.1. Wait, yes. So the equation sim

Step,Training Loss,reward,reward_std,completions / mean_length,completions / min_length,completions / max_length,completions / clipped_ratio,completions / mean_terminated_length,completions / min_terminated_length,completions / max_terminated_length,kl,rewards / match_format_exactly / mean,rewards / match_format_exactly / std,rewards / match_format_approximately / mean,rewards / match_format_approximately / std,rewards / check_answer / mean,rewards / check_answer / std,rewards / check_numbers / mean,rewards / check_numbers / std
1,0.000300,-2.031250,3.477078,1619.125000,963.000000,1849.000000,0.562500,1323.571533,963.000000,1561.000000,0.323402,1.312500,1.537043,-0.281250,2.205816,-1.500000,1.974842,-1.562500,1.436141
2,0.000300,-3.500000,4.568914,1678.500000,792.000000,1849.000000,0.562500,1459.285767,792.000000,1779.000000,0.341252,1.312500,1.537043,-1.031250,2.305564,-1.718750,1.277612,-2.062500,0.512348
3,0.000300,-6.125000,2.750000,1771.562500,1308.000000,1849.000000,0.812500,1436.000000,1308.000000,1570.000000,0.293771,0.562500,1.209339,-2.156250,1.814008,-2.218750,0.631631,-2.312500,0.403113
4,0.000300,-6.500000,2.000000,1837.687500,1755.000000,1849.000000,0.875000,1758.500000,1755.000000,1762.000000,0.289642,0.375000,1.024695,-2.437500,1.537043,-2.062500,0.170783,-2.375000,0.341565
5,0.000400,-2.000000,3.913006,1623.125000,887.000000,1849.000000,0.375000,1487.599976,887.000000,1815.000000,0.358510,1.875000,1.500000,-0.187500,2.250000,-1.812500,1.314978,-1.875000,0.500000
6,0.000300,-2.437500,5.172111,1656.937500,972.000000,1849.000000,0.500000,1464.875000,972.000000,1775.000000,0.287873,1.500000,1.549193,-0.750000,2.323790,-1.812500,1.611159,-1.375000,1.962142
7,0.000300,-5.375000,2.896802,1682.125000,1098.000000,1849.000000,0.687500,1315.000000,1098.000000,1564.000000,0.292910,0.937500,1.436141,-1.593750,2.154211,-2.531250,0.991106,-2.187500,0.478714
8,0.000300,-2.093750,3.918193,1524.125000,919.000000,1849.000000,0.375000,1329.200073,919.000000,1823.000000,0.277257,1.687500,1.537043,-0.281250,2.205816,-1.937500,1.948290,-1.562500,1.436141
9,0.000400,-3.000000,5.199610,1517.125000,974.000000,1849.000000,0.500000,1185.250000,974.000000,1393.000000,0.361153,1.500000,1.549193,-0.656250,2.256241,-2.156250,2.134782,-1.687500,1.470544
10,0.000300,-6.000000,3.000000,1767.312500,1278.000000,1849.000000,0.812500,1413.333374,1278.000000,1503.000000,0.263243,0.562500,1.209339,-2.156250,1.814008,-2.093750,0.201556,-2.312500,0.403113


TrainOutput(global_step=100, training_loss=0.0003169380864710547, metrics={'train_runtime': 5726.456, 'train_samples_per_second': 0.279, 'train_steps_per_second': 0.017, 'total_flos': 0.0, 'train_loss': 0.0003169380864710547})

<a name="Inference"></a>
### Inference
现在让我们来测试一下刚刚训练好的模型！首先，我们先测试一下没有训练任何GRPO的模型：

In [32]:
text = "What is the sqrt of 101?"

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 1.0,
    top_k = 50,
    max_tokens = 1024,
)
output = model.fast_generate(
    [text],
    sampling_params = sampling_params,
    lora_request = None,
)[0].outputs[0].text

output

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

" I use the online calculator function with my Windows computer on a 64-bit version of Java. The online calculator function provides two results, one that is positive and another that is negative, but I'm surprised the calculator function gives a positive result for the square root of 101.\n\n## Step 1: Review the math problem\nThe problem is to find the square root of 101 without using a calculator. This requires understanding the math properties of square roots.\n\n## Step 2: Identify the relevant property of square roots\nOne of the key properties of square roots is that the square root of a number can be positive or negative, depending on whether the number is on the left side of the number line or the right side of the number. This means we can have a positive square root and a negative square root.\n\n## Step 3: Use the square root property to find the square root of 101\nSince 100 is the closest perfect square that is less than 101, we can use this difference to find the square 

现在有了我们刚刚用 GRPO 训练的 LoRA，我们先保存 LoRA！

In [33]:
model.save_lora("grpo_saved_lora")

验证 LoRA 是否已完成训练！

In [34]:
from safetensors import safe_open

tensors = {}
with safe_open("grpo_saved_lora/adapter_model.safetensors", framework = "pt") as f:
    # Verify both A and B are non zero
    for key in f.keys():
        tensor = f.get_tensor(key)
        n_zeros = (tensor == 0).sum() / tensor.numel()
        assert(n_zeros.item() != tensor.numel())

现在我们加载LoRA并进行测试：

In [35]:
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user",   "content": "What is the sqrt of 101?"},
]

text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
    tokenize = False,
)
from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 1.0,
    top_k = 50,
    max_tokens = 2048,
)
output = model.fast_generate(
    text,
    sampling_params = sampling_params,
    lora_request = model.load_lora("grpo_saved_lora"),
)[0].outputs[0].text

output

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

"Okay, let's see. The problem is asking for the square root of 101. Hmm, square root, right, so that's something you take the number between the square brackets, like, √x. So I need to find the number that when multiplied by itself gives 101.\n\nLet me check some numbers. 10 squared is 100, so that's too low. 11 squared is 121, which is much bigger than 101. So 11 is the square root of 101. Wait, let me make sure. Let me write it down again: 11×11=121, yeah, that's right. So the square root of 101 is 11. That makes sense.\n\nBut just to confirm, maybe there's another way to think about it. Wait, the question is asking for the square root, so I think 11 is the answer. Because the square root of 101 is the number that, when you multiply it by itself, gives 101. I don't think there's another number that does that. 11 is the only number that when squared equals 101. So that's why 11.\nTo find the square root of 101, we proceed as follows:\n\n1. We know that:\n   \\[\n   \\sqrt{101} = x\n  

我们的推理模型好多了——虽然它并不总是正确的，因为我们只训练了一个小时左右——如果我们增加序列长度并进行更长时间的训练，情况会更好！

<a name="Save"></a>
### 保存为 float16 以用于 VLLM

我们也支持直接保存为 `float16` 格式。选择 `merged_16bit` 保存为 `float16` 格式，选择 `merged_4bit` 保存为 `int4` 格式。我们还允许使用 `lora` 适配器作为备用方案。使用 `push_to_hub_merged` 上传到 Hugging Face 帐户！

In [36]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False:
    model.save_pretrained("model")
    tokenizer.save_pretrained("model")
if False:
    model.push_to_hub("hf/model", token = "")
    tokenizer.push_to_hub("hf/model", token = "")


### GGUF / llama.cpp 转换

我们现在原生支持保存为 `GGUF` / `llama.cpp` 格式！我们会克隆 `llama.cpp`，并默认将其保存为 `q8_0`。我们支持所有类似 `q4_k_m` 的方法。使用 `save_pretrained_gguf` 进行本地保存，使用 `push_to_hub_gguf` 上传到 HF。

支持的量化方法（完整列表请参见我们的[Wiki页面](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)）：

* `q8_0` - 转换速度快。资源占用高，但通常可以接受。

* `q4_k_m` - 推荐。对 attention.wv 和 feed_forward.w2 张量的一半使用 Q6_K，其余使用 Q4_K。

* `q5_k_m` - 推荐。对 attention.wv 和 feed_forward.w2 张量的一半使用 Q6_K，其余使用 Q5_K。

In [37]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "hf/model", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "",
    )

现在，请在 llama.cpp 中使用 `model-unsloth.gguf` 文件或 `model-unsloth-Q4_K_M.gguf` 文件。